In [1]:
import wikipediaapi
from __future__ import division
from sklearn.cluster import KMeans
from numbers import Number
from pandas import DataFrame
import sys, codecs, numpy
import numpy as np
import argparse
import logging
import time
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [2]:
wiki_wiki = wikipediaapi.Wikipedia('en')
page = wiki_wiki.page('Berlin')

In [3]:
def print_sections(sections, level=0):
        for s in sections:
                print("%s: %s - %s" % ("*" * (level + 1), s.title, s.text[0:40]))
                print_sections(s.sections, level + 1)

In [4]:
print_sections(page.sections)

*: History - 
**: Etymology - Berlin lies in northeastern Germany, eas
**: 12th to 16th centuries - The earliest evidence of settlements in 
**: 17th to 19th centuries - The Thirty Years' War between 1618 and 1
**: 20th to 21st centuries - In the early 20th century, Berlin had be
*: Geography - 
**: Topography - Berlin is situated in northeastern Germa
**: Climate - Berlin has a Maritime temperate climate 
**: Cityscape - Berlin's history has left the city with 
**: Architecture - The Fernsehturm (TV tower) at Alexanderp
*: Demographics - On 30 June 2017 the city-state of Berlin
**: Nationalities - National and international migration int
**: Languages - German is the official and predominant s
**: Religion - More than 60% of Berlin residents have n
*: Government - 
**: City state - Since the reunification on 3 October 199
**: Boroughs - Berlin is subdivided into 12 boroughs or
**: Twin towns – sister cities - Berlin maintains official partnerships w
**: Capital city - Berlin is the ca

In [5]:
section_title=[]
for s in page.sections:
    section_title.append(s.title)
section_title

['History',
 'Geography',
 'Demographics',
 'Government',
 'Economy',
 'Infrastructure',
 'Education',
 'Culture',
 'See also',
 'Notes',
 'References',
 'External links']

In [6]:
index = section_title.index('History')
for sub in page.sections[index].sections:
    print(sub.title)
    print(sub.text)
    print('#'*50)

Etymology
Berlin lies in northeastern Germany, east of the river Elbe and its tributary (Saxon/Thuringian) Saale, which once constituted the eastern border of the Frankish Realm. While the Frankish Realm was primarily inhabited by Germanic tribes like the Franks and the Saxons, the regions east of the border rivers were inhabited by Slavic tribes. This is why most of the cities and villages in northeastern Germany have Slavic-derived names (Germania Slavica). Typical German place name endings of Slavic origin are -ow, -itz, -itzsch and -in. The name Berlin has its roots in the language of West Slavic inhabitants of the area of today's Berlin, and may be related to the Old Polabian stem berl-/birl- ("swamp"). There are many boroughs of Slavic origin in the city: Berlin-Karow, Berlin-Malchow, Berlin-Pankow, Berlin-Spandau (earlier: Spandow), Berlin-Gatow, Berlin-Kladow, Berlin-Steglitz, Berlin-Lankwitz, Berlin-Britz, Berlin-Buckow, Berlin-Rudow, Berlin-Alt-Treptow, Berlin-Schmöckwitz, Be

In [7]:
start = time.time()
print("Load word2vec model ... ", end="", flush=True)
model_path='data/GoogleNews-vectors-negative300.bin'
w2v_model = KeyedVectors.load_word2vec_format(model_path, binary=bool(1))
print("finished in {:.2f} sec.".format(time.time() - start), flush=True)
word_vectors = w2v_model.wv.syn0
n_words = word_vectors.shape[0]
vec_size = word_vectors.shape[1]
print("#words = {0}, vector size = {1}".format(n_words, vec_size))

Load word2vec model ... finished in 59.17 sec.
#words = 3000000, vector size = 300


In [8]:
# get the labels, and vectors for each label
array=[]
with open('data/all_predicate.txt', 'r') as myfile:
    array=[]
    for line in myfile:
        *others, last = line.split('/')
        last=last.rstrip('\n') 
        array.append(last)
        
with open('data/all_predicate_entity.txt', mode='wt', encoding='utf-8') as myfile:
    myfile.write('\n'.join(array))


In [9]:
from utils import *
from wordCluster import *

In [13]:
labels_array = []
numpy_arrays = []
for entity in array:
    labels_array.append(entity)
    sr = phrase_vector(entity, w2v_model)
    numpy_arrays.append(np.array([float(i) for i in sr[0:]]) )
numpy_arrays = np.array( numpy_arrays )

numOfCluster=52
print("clustering in {} groups \n".format(numOfCluster))
kmeans_model = KMeans(init='k-means++', n_clusters=numOfCluster, n_init=10)
kmeans_model.fit(numpy_arrays)
cluster_labels  = kmeans_model.labels_
cluster_inertia   = kmeans_model.inertia_
cluster_to_words  = find_word_clusters(labels_array, cluster_labels)


clustering in 52 groups 



In [ ]:
most_similar_words=[]
for i in range(0, numOfCluster):
    #print(i)
    model_word_vector = np.array(kmeans_model.cluster_centers_[i], dtype='f')
    similar_word = w2v_model.most_similar( [ model_word_vector ], [], topn=1)[0][0]
    
    most_similar_words.append(similar_word)

In [23]:
for c in range(0, numOfCluster):
    print(most_similar_words[c])
    print(cluster_to_words[c])
    print("\n")

broadcast
['BroadcastNetwork', 'Presenter', 'ProgrammingLanguage', 'RadioHost', 'RadioProgram', 'RadioStation', 'Stream', 'TelevisionEpisode', 'TelevisionHost', 'TelevisionSeason', 'TelevisionShow', 'TelevisionStation', 'analogChannel', 'broadcastArea', 'broadcastNetwork', 'broadcastRepeater', 'broadcastStationClass', 'broadcastTranslator', 'channel', 'digitalChannel', 'digitalSubChannel', 'format', 'formerBroadcastNetwork', 'formerChannel', 'guest', 'network', 'otherChannel', 'presenter', 'programmeFormat', 'programmingLanguage', 'series', 'show', 'televisionSeries', 'timeshiftChannel', 'virtualChannel', 'voice', 'webcast']


play
['Game', 'Play', 'VideoGame', 'actScore', 'battingSide', 'characterInPlay', 'chorusCharacterInPlay', 'field', 'firstGame', 'gameEngine', 'games', 'muteCharacterInPlay', 'plays', 'satScore', 'settingOfPlay', 'shoots', 'subjectOfPlay']


protein
['Archaea', 'Bacteria', 'Brain', 'Enzyme', 'Eukaryote', 'HumanGene', 'HumanGeneLocation', 'MouseGene', 'MouseGeneLoc

In [ ]:
question = 'where do I study'
keyword='education'
word_vector=phrase_vector(keyword, w2v_model)
label = kmeans_model.predict(word_vector)[0]
clusterWord = most_similar_words[label]
clusterWord

In [347]:
page = wiki_wiki.page('Bonn')
#print_sections(page.sections)

In [342]:
def getTopSection(sections, clusterWord):
    section_title=[]
    for s in sections:
        section_title.append([s.title, phrase_similarity(clusterWord, s.title)])
    simi_list = [i[1] for i in section_title]
    index = simi_list.index(max(simi_list))
    selected_section = sections[index]
    text = sections[index].text
    return selected_section, text

In [350]:
selected_section, text_section =getTopSection(page.sections, clusterWord)
print(selected_section.title, "section has", len(selected_section.sections), "subsections")
print('='*50)
summary = []
if len(selected_section.sections)<=1:
    summary.append(page.summary)
    summary.append(text_section)
    
else:
    selected_subsection, text_subsection= getTopSection(selected_section.sections, keyword)
    summary.append(page.summary)
    summary.append(text_subsection)
print("summary for page: ")
print(summary[0])
print('-'*50)
print("summary for subsection: ", selected_section.title )
print(summary[1])

Education section has 1 subsections
summary for page: 
The Federal City of Bonn (German pronunciation: [ˈbɔn] ( listen)) is a city on the banks of the Rhine in the German state of North Rhine-Westphalia, with a population of over 300,000. About 24 km (15 mi) south-southeast of Cologne, Bonn is in the southernmost part of the Rhine-Ruhr region, Germany's largest metropolitan area, with over 11 million inhabitants.
Because of a political compromise following German reunification, the German state maintains a substantial presence in Bonn. Bonn is the secondary seat of the President, the Chancellor, the Bundesrat and the primary seat of six federal government ministries and twenty federal authorities. The unique title of Federal City (German: Bundesstadt) reflects its important political status within Germany.
Founded in the 1st century BC as a Roman settlement, Bonn is one of Germany's oldest cities. From 1597 to 1794, Bonn was the capital of the Electorate of Cologne, and residence of th